# Using HackerNews API

## Week 8. Practice Programming Assignment 1

In this assignment we going to work with [HackerNews](https://news.ycombinator.com/). You are required to use its [API](https://github.com/HackerNews/API) to get all items from 2007-04-01 00:01 to 2007-04-07 23:59, and then use the data to answer questions you will find below.

###### Import coursera grader tools

In [ ]:
import sys
sys.path.append("..")


1175385601 Start time: Sun, 01 Apr 2007 00:00:01 GMT

1175990340 End time: Sat, 07 Apr 2007 23:59:00 GMT

In [ ]:
1175990070

<br><br><br><br>

### Coding part

In [112]:
import requests
id = 886347
url = f" https://hacker-news.firebaseio.com/v0/item/{id}.json"

response = requests.get(url)
print(response.status_code)
print(bool(response.json()))
r = response.json()
print(r['by'])
print(r['time'],
r.get('id'),
r.get('type'),
r.get('by'),
r.get('time'),
r.get('dead'),
r.get('parent'),
r.get('poll'),
r.get('kids'),
r.get('url'),
r.get('score'),
r.get('title'),
r.get('parts'),
r.get('descendants'))

200
True
gwern
1255734092 886347 comment gwern 1255734092 None 886283 None None None None None None None


In [113]:
r.get('text')

"&#62; Why would you want to record your whole life, especially on a pendant-cam that would be bouncy and pointing at something irrelevant half the time.<p>Family videos are bouncy and pointing at something irrelevant half the time as well.<p>&#62; Do you keep an audio record of your whole life now? That's been feasible for a long time. Why not? What about video would make you start?<p>Omni-directional mikes would pick up too much; directional would be inconvenient to aim. Video is vastly easier to navigate, since we think spatially and visually - how do you scroll through an audio recording? With video, you get feedback as to where you are temporally, especially if there's movement from place to place; with audio, you just randomly fastforward hoping to get nearer where you want to hear, if you can even recognize progress towards or away from the clip you want. (An example: I have a video of a boardwalk, versus audio, and I want to look at an encounter. Will it be easier for me to fin

In [19]:
response.json()

32178031

In [15]:
max_id_url = "https://hacker-news.firebaseio.com/v0/maxitem.json?print=pretty"
response = requests.get(max_id_url)
assert response.status_code == 200
max_id = response.json()
print(max_id)

32177911


In [118]:
import time
from tqdm.notebook import tqdm
import datetime
import pandas as pd
 

def parse_HN(start_id):
    hnews = []
    for id in tqdm(range(start_id, 0, -1)):
        time.sleep(0.01)
        url = f" https://hacker-news.firebaseio.com/v0/item/{id}.json"
        response = requests.get(url)
        r = response.json()
        assert response.status_code == 200, f" This id broke the thing: {id}"
        
        if r:
            d = {}
            d['id'] = r.get('id')
            d['type'] = r.get('type')
            d['by'] = r.get('by')
            d['time'] = r.get('time')
            d['dead'] = r.get('dead')
            d['parent'] = r.get('parent')
            d['poll'] = r.get('poll')
            d['kids'] = r.get('kids')
            d['url'] = r.get('url')
            d['score'] = r.get('score')
            d['title'] = r.get('title')
            d['text'] = r.get('text')
            d['parts'] = r.get('parts')
            d['descendants'] = r.get('descendants')
            d['len_text'] = len(r.get('text')) if r.get('text') else 0
            d['len_kids'] = len(r.get('kids')) if r.get('kids') else 0
            hnews.append(d)
            
            
            #print(r.get('id'),r.get('time'), 
            #datetime.datetime.utcfromtimestamp(r.get('time')).strftime('%Y-%m-%dT%H:%M:%SZ'))
            if r.get('time') < 1175385601:
                break
        else:
            print('No content')
    return  pd.DataFrame(hnews)


In [119]:
df = parse_HN(10235)

  0%|          | 0/10235 [00:00<?, ?it/s]

In [120]:
df.to_csv('hacker_news.csv', index=False)
df.head()

,id,type,by,time,dead,parent,poll,kids,url,score,title,text,parts,descendants,len_text,len_kids
0,10235,comment,danw,1175990482,None,9986.0,None,None,None,NaN,None,"Bristol, UK; 2; 21, 20",None,NaN,22,0
1,10234,comment,rolodex,1175990070,None,10140.0,None,[10244],None,NaN,None,I have no idea what you're talking about. To m...,None,NaN,224,1
2,10233,story,danw,1175989806,None,NaN,None,None,http://www.zephoria.org/thoughts/archives/2007...,6.0,"""Demo or die"" - 5 secrets to success",None,None,0.0,0,0
3,10232,story,danw,1175989725,None,NaN,None,[10296],http://www.webkitchen.co.uk/2006/03/startup.html,3.0,"""Shielded in a cloak of ignorance and enthusia...",None,None,1.0,0,1
4,10231,story,kevinxray,1175989395,None,NaN,None,None,http://www.allbusiness.com/marketing-advertisi...,2.0,Lessons from Greatness,None,None,0.0,0,0


In [145]:
print(df.shape)
df2 = df[(df["time"] > 1175385601) & (df["time"] < 1175990340)].copy()
print(df2.shape)
df2["r_time"] = pd.to_datetime(df2['time'],unit='s')
df2["dow"] = df2["r_time"].dt.day_name()
df2["doy"] = df2["r_time"].dt.day_of_year
df2.head(20)

(2320, 16)
(2318, 16)


,id,type,by,time,dead,parent,poll,kids,url,score,title,text,parts,descendants,len_text,len_kids,r_time,dow,doy
1,10234,comment,rolodex,1175990070,None,10140.0,None,[10244],None,NaN,None,I have no idea what you're talking about. To m...,None,NaN,224,1,2007-04-07 23:54:30,Saturday,97
2,10233,story,danw,1175989806,None,NaN,None,None,http://www.zephoria.org/thoughts/archives/2007...,6.0,"""Demo or die"" - 5 secrets to success",None,None,0.0,0,0,2007-04-07 23:50:06,Saturday,97
3,10232,story,danw,1175989725,None,NaN,None,[10296],http://www.webkitchen.co.uk/2006/03/startup.html,3.0,"""Shielded in a cloak of ignorance and enthusia...",None,None,1.0,0,1,2007-04-07 23:48:45,Saturday,97
4,10231,story,kevinxray,1175989395,None,NaN,None,None,http://www.allbusiness.com/marketing-advertisi...,2.0,Lessons from Greatness,None,None,0.0,0,0,2007-04-07 23:43:15,Saturday,97
5,10230,story,kevinxray,1175989236,None,NaN,None,[10396],http://blogs.zdnet.com/micro-markets/?p=1191,2.0,"Why Google IS afraid of Microsoft, big time",None,None,1.0,0,1,2007-04-07 23:40:36,Saturday,97
6,10229,story,kevinxray,1175989110,None,NaN,None,[10269],http://www.costar.com/News/Article.aspx?id=AE4...,1.0,Microsoft Expanding By 1 Million SF in Bellevue,None,None,1.0,0,1,2007-04-07 23:38:30,Saturday,97
7,10228,comment,jward,1175988936,None,9986.0,None,None,None,NaN,None,"Edmonton, Alberta, Canada;2;26,28",None,NaN,33,0,2007-04-07 23:35:36,Saturday,97
8,10227,comment,bartsimpson007,1175988354,None,10013.0,None,None,None,NaN,None,"To rephrase, does Paul suggest that Apple is d...",None,NaN,146,0,2007-04-07 23:25:54,Saturday,97
9,10226,comment,randallsquared,1175988265,None,10113.0,None,None,None,NaN,None,I think not selecting anyone would be bad for ...,None,NaN,249,0,2007-04-07 23:24:25,Saturday,97
10,10225,story,danw,1175988171,None,NaN,None,None,http://maps.google.co.uk/maps/ms?ie=UTF8&hl=en...,1.0,London Valley Map - Google MyMap of London sta...,None,None,0.0,0,0,2007-04-07 23:22:51,Saturday,97


In [123]:
print(df2.id.max(), df2.id.min(), len(df2.id.values))

10234 7917 2318


<br><br>

### Questions

<br><br>

**Question 1.** How many stories were created during the period??

In [125]:
df2[df2['type']=='story'].shape[0]

487

In [ ]:
answer_part_1 = 487

<br>

**Question 2.** What was the length of the longest story?

In [201]:
int(df2[df2['type']=='story']['descendants'].max())

445

In [ ]:
answer_part_2 = 445

<br>

**Question 3.** What user left the most comments?

In [133]:
print(df2[df2['type']=='comment'].groupby('by')['id'].count()['pg'])
print(df2[df2['type']=='comment'].groupby('by')['id'].count().max())
print(df2[df2['type']=='comment'].groupby('by')['id'].count().idxmax())

62
62
pg


In [130]:
df2[df2['type']=='comment'].groupby('by')['id'].count().idxmax()

'pg'

In [ ]:
answer_part_3 = 'pg'

In [ ]:
# Setting our answers to grader. Do not change!
grader.set_answer("QQkMj", answer_part_3)

<br>

**Question 4.** What user created the most stories?

In [135]:
print(df2[df2['type']=='story'].groupby('by')['id'].count()['usablecontent'])
print(df2[df2['type']=='story'].groupby('by')['id'].count().max())
print(df2[df2['type']=='story'].groupby('by')['id'].count().idxmax())

34
34
usablecontent


In [ ]:
answer_part_4 = "usablecontent"

In [ ]:
# Setting our answers to grader. Do not change!
grader.set_answer("CNPy6", answer_part_4)

<br>

**Question 5.** What is day of the week with the most stories created? Enter one of the following as an answer: Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday.

In [139]:
print(df2[df2['type']=='story'].groupby('dow')['id'].count().idxmax())
df2[df2['type']=='story'].groupby('dow')['id'].count()

Wednesday


dow
Friday       68
Monday       88
Saturday     53
Sunday       38
Thursday     67
Tuesday      81
Wednesday    92
Name: id, dtype: int64

In [ ]:
answer_part_5 = "Wednesday"

<br><br>

**Question 6.** What is day of the week with the most comments left? Enter one of the following as an answer: Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday.

In [140]:
print(df2[df2['type']=='comment'].groupby('dow')['id'].count().idxmax())
df2[df2['type']=='comment'].groupby('dow')['id'].count()

Thursday


dow
Friday       250
Monday       201
Saturday     410
Sunday       124
Thursday     414
Tuesday      194
Wednesday    238
Name: id, dtype: int64

In [ ]:
answer_part_6 = "Thursday"

<br>

**Question 7.** How many comments does user leave on average?

In [142]:
print(df2[df2['type']=='comment'].groupby('by')['id'].count().mean())
df2[df2['type']=='comment'].groupby('by')['id'].count()

4.755844155844156


by
Alex3917    13
AurSaraf     1
BearOfNH     1
Benja        3
BitGeek     12
            ..
yc1          1
zach        21
zaidf       47
zkinion     24
zzzeek       1
Name: id, Length: 385, dtype: int64

In [ ]:
answer_part_7 = 4.755844155844156

<br>

**Question 8.** What is a median comments per user?

In [143]:
print(df2[df2['type']=='comment'].groupby('by')['id'].count().median())
df2[df2['type']=='comment'].groupby('by')['id'].count()

2.0


by
Alex3917    13
AurSaraf     1
BearOfNH     1
Benja        3
BitGeek     12
            ..
yc1          1
zach        21
zaidf       47
zkinion     24
zzzeek       1
Name: id, Length: 385, dtype: int64

In [ ]:
answer_part_8 = 2

<br>

**Question 9.** What is an average number of stories per day?

In [149]:
df2.doy.nunique()

7

In [147]:
print(df2[df2['type']=='story'].groupby('doy')['id'].count().mean())
df2[df2['type']=='story'].groupby('doy')['id'].count()

69.57142857142857


doy
91    38
92    88
93    81
94    92
95    67
96    68
97    53
Name: id, dtype: int64

In [ ]:
answer_part_9 = 69.57142857142857

<br>

**Question 10.** How many comments per day are left on average?

In [148]:
print(df2[df2['type']=='comment'].groupby('doy')['id'].count().mean())
df2[df2['type']=='comment'].groupby('doy')['id'].count()

261.57142857142856


doy
91    124
92    201
93    194
94    238
95    414
96    250
97    410
Name: id, dtype: int64

In [ ]:
answer_part_10 = 261.57142857142856

<br>
<br>

### Submitting answers

In [ ]:
# you can make submission with answers so far to check yourself at this stage
grader.submit(COURSERA_EMAIL, COURSERA_TOKEN)